### 初期設定

In [ ]:
# --- Notebook初期設定 ---
%load_ext autoreload
%autoreload 2
from utils.data_loarder import load_csv_to_db, get_df_from_db
import utils.config as cfg
import utils.data_visualize as dv
import utils.data_pivot_from_DB as dp
print("🔁 autoreload 有効化完了")
import os
import pandas as pd
import plotly.express as px
import yaml

# 設定読み込み
settings = cfg.load_settings("setting.yaml")
display(settings)

In [ ]:
csv_files = {}
for file in settings["files"]["raw"]:
    csv_files[settings["files"]["raw"][file]] = os.path.join(settings["data_path"]["raw"],"",settings["files"]["raw"][file]) 
#print(csv_files)
DB_PATH_FINANCE = os.path.join(settings["database_path"],"",settings["database"]["finance"])

load_csv_to_db(
    csv_files=csv_files,
    db_path=DB_PATH_FINANCE
)

### 動作確認

#### _________ダッシュボード

In [ ]:
graphs = []   
# 1. 総資産推移
df = get_df_from_db(DB_PATH_FINANCE, "資産詳細", "date", None,["資産額"],"sum",set_index=True)

end_date = df.index.max() + pd.DateOffset(months=1)
where_clause = f"date <= DATE('{end_date}')"
df_target = get_df_from_db(DB_PATH_FINANCE, "資産目標値", "date", None,["資産額"],"sum",where_clause=where_clause,set_index=True)

df_merged = pd.merge(df, df_target, left_index=True,right_index=True, how="outer", suffixes=("_実績", "_目標"))

fig = px.line(df_merged, x=df_merged.index, y=df_merged.columns, title="総資産推移", labels={"date": "日付", "values":"資産額","variable":""}, template="plotly_dark")
fig.update_xaxes(tickformat="%y/%m/%d")
fig.update_layout(autosize=True, margin=dict(l=10,r=10,t=30,b=30))
#fig.show()

# 2. トータルリターン推移
df = get_df_from_db(DB_PATH_FINANCE, "資産詳細", "date", None, ["トータルリターン"],"sum",set_index=True)

end_date = df.index.max() + pd.DateOffset(months=1)
where_clause = f"date <= DATE('{end_date}')"
df_target = get_df_from_db(DB_PATH_FINANCE, "資産目標値", "date", None, ["トータルリターン"],"sum",where_clause=where_clause,set_index=True)
df_target["トータルリターン"] = df_target["トータルリターン"].cumsum()

df_merged = pd.merge(df, df_target, left_index=True,right_index=True, how="outer", suffixes=("_実績", "_目標"))

fig = px.line(df_merged, x=df_merged.index, y=df_merged.columns, title="トータルリターン推移", labels={"date": "日付", "values":"トータルリターン","variable":""}, template="plotly_dark")
fig.update_xaxes(tickformat="%y/%m/%d")
fig.update_layout(autosize=True, margin=dict(l=10,r=10,t=30,b=30))
#fig.show()

# 3～6 収支
df_balance = get_df_from_db(DB_PATH_FINANCE, "収支詳細", index_col="date", columns_col=["収支タイプ","収支カテゴリー"], values_col=["金額","目標"], aggfunc="sum",set_index=True)
# 3～4 一般
df_general = df_balance.query('収支タイプ == "一般収支"')
# 3. 一般収入・支出推移
df_pivot = df_general.pivot_table(index='date', columns='収支カテゴリー', values=['金額', '目標'], aggfunc='sum')
df_pivot.columns = [f"{val}_{cat}" for val, cat in df_pivot.columns]
df_monthly = df_pivot.resample('ME').sum()
fig = px.bar(
    df_monthly, x=df_monthly.index, y=['金額_収入', '金額_支出'], barmode='group', template='plotly_dark',
    labels={'value':'金額', 'date':'年月', 'variable':'種別'}
)
fig.add_scatter(
    df_monthly,
    x=df_monthly.index,
    y=df_monthly['目標_収入'],
    mode='lines+markers',
    #name='目標収入',
    #line=dict(color='blue', dash='dash')
)

fig.show()

df_monthly
# 4. 一般収支推移

# 5～6 特別
# 5. 特別収入：支出推移
# 6. 特別収支推移

# 4. 一般収支推移


# 6. 任意グラフ（後で追加可能）

ValueError: 
    Invalid value of type 'pandas.core.frame.DataFrame' received for the 'alignmentgroup' property of scatter
        Received value:                目標_収入     目標_支出     金額_収入     金額_支出
date                                              
2024-10-31  357000.0  264567.0  357011.0  259169.0
2024-11-30  357000.0  264567.0  386353.0  233799.0
2024-12-31  357000.0  264567.0  364462.0  144488.0
2025-01-31  357000.0  264567.0  366845.0  259620.0
2025-02-28  357000.0  264567.0  354905.0  248408.0
2025-03-31  357000.0  264567.0  451744.0  226707.0
2025-04-30  357000.0  274567.0  407282.0  269775.0
2025-05-31  357000.0  274567.0  368109.0  199515.0
2025-06-30  357000.0  274567.0  425437.0  227956.0
2025-07-31  357000.0  274567.0  450180.0  222419.0
2025-08-31  357000.0  274567.0  527916.0  278589.0
2025-09-30  357000.0  274567.0  295103.0  296227.0
2025-10-31       0.0       0.0    5701.0    5960.0

    The 'alignmentgroup' property is a string and must be specified as:
      - A strin

,date,収支タイプ,収支カテゴリー,金額,目標
0,2024-10-01,一般収支,収入,0.0,0.0
1,2024-10-01,一般収支,支出,0.0,0.0
2,2024-10-01,特別収支,収入,0.0,0.0
3,2024-10-01,特別収支,支出,0.0,0.0
4,2024-10-02,一般収支,収入,0.0,0.0
...,...,...,...,...,...
1547,2025-10-22,特別収支,支出,0.0,0.0
1548,2025-10-23,一般収支,収入,126.0,0.0
1549,2025-10-23,一般収支,支出,0.0,0.0
1550,2025-10-23,特別収支,収入,0.0,0.0


### SQLite関連

In [ ]:
conn = sqlite3.connect(DB_PATH_FINANCE)
cur = conn.cursor()

# .csv付きのテーブルを削除
for table in ["資産詳細.csv", "収支詳細.csv", "資産目標値.csv"]:
    cur.execute(f'DROP TABLE IF EXISTS "{table}"')
    print(f"削除: {table}")

conn.commit()
conn.close()

In [ ]:
conn = sqlite3.connect(DB_PATH_FINANCE)
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
conn.close()

print(tables)